# Importation des librairies

In [1]:
import pandas as pd
import numpy as np

import missingno as msno

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from scipy.stats import zscore
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


# Importation des données

In [2]:
df = pd.read_csv("dataset_assurance.csv")


In [ ]:
df

### Descriptif des variables

- **Indice de masse corporel (BMI)** : Permet de donner un rapport entre la taille et le poids. Idéalement, il faut être entre 18.5 et 24.9.
- **Sexe (Sex)** : Genre de la personne qui contracte l’assurance (homme ou femme).
- **Âge (Age)** : Âge du principal bénéficiaire.
- **Nombre d’enfants à charge (Children)** : Nombre d’enfants couverts par l’assurance.
- **Fumeur (Smoker)** : Fumeur ou non-fumeur.
- **Région (Region)** : Zone résidentielle dans les États-Unis (Nord-Est, Sud-Est, Sud-Ouest, Nord-Ouest).
- **Charges (Charges)** : La prime d’assurance facturée (cible).

# I | Exploration et Analyse

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

## 1. Valeurs dupliquées & manquantes

In [ ]:
### Valeurs dupliquées / doublons

# Identification des doublons
doublons = df[df.duplicated(keep=False)]
print(doublons)

""" Il y a en effet une ligne en doublon, nous allons dans ce cas la supprimer (peu importe 1ère(first) ou 2nd itération(last))"""

In [8]:
# Suppression des doublons
df.drop_duplicates(inplace=True)

In [ ]:
### Valeurs manquantes

df.isna().sum()

In [ ]:
msno.matrix(df)

## 2. Détection des valeurs aberrantes

In [ ]:
### Boîtes à moustache de nos variables quantitatives pour déterminer les valeurs aberrantes

fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# Graphique 1 : Distribution des charges
sns.boxplot(x=df['charges'], ax=axes[0])
axes[0].set_title("Distribution des Charges")
axes[0].set_xlabel("Charges")
axes[0].set_ylabel("Nombre d'assurés")

# Graphique 2 : Distribution des âges
sns.boxplot(x=df['age'], ax=axes[1])
axes[1].set_title("Distribution des Âges")
axes[1].set_xlabel("Âge")
axes[1].set_ylabel("")


# Graphique 3 : Distribution du BMI
sns.boxplot(x=df['bmi'], ax=axes[2])
axes[2].set_title("Distribution du BMI")
axes[2].set_xlabel("BMI")
axes[2].set_ylabel("")

for i in range(3):
    axes[i].grid()
    
plt.show()

In [12]:
### Recherches des valeurs aberrantes

def valeurs_aberrantes_IQR(df, colonne) : 
    Q1 = df[colonne].quantile(0.25)
    Q3 = df[colonne].quantile(0.75)
    IQR = Q3 - Q1

    limite_inf = Q1 - (1.5 * IQR)
    limite_sup = Q3 + (1.5 * IQR)

    valeurs_aberrantes = df[(df[colonne] < limite_inf) | (df[colonne] > limite_sup)]
    
    return valeurs_aberrantes

In [ ]:
# Valeurs aberrantes de la variable cible "charges"

charges_outliers = valeurs_aberrantes_IQR(df, 'charges')
charges_outliers.sort_values('charges', ascending=False)

In [ ]:
print(f"- Nombre de lignes contenant des valeurs abérantes : {charges_outliers.shape[0]}")
print(f"- Pourgentage de lignes contenant des valeurs abérantes : {round((charges_outliers.shape[0] / df.shape[0]) * 100,2)} %")

In [ ]:
# Vérification des données : Age
print("Vérification des données : Âge")
print(f"Valeur minimale : {min(df['age'])}")
print(f"Valeur maximale : {max(df['age'])}")
print(f"Type de données : {df['age'].dtype}\n")

print("Observation : Les données sur l'âge semblent correctes et ne contiennent pas de valeurs aberrantes.\n")

# Vérification des données : BMI
print("Vérification des données : BMI")
print(f"Valeur minimale : {min(df['bmi']):.2f}")
print(f"Valeur maximale : {max(df['bmi']):.2f}\n")

print("Observation : Bien qu'il n'existe pas de limites strictes au BMI, une valeur inférieure à 16 indique une insuffisance pondérale sévère, et une valeur supérieure à 40 représente une obésité de classe 3 (sévère).")
print("Nous pourrions donc considérer que des valeurs en dehors de cet intervalle sont potentiellement aberrantes.\n")

# Vérification des données : Nombre d'enfants
print("Vérification des données : Nombre d'enfants")
print(f"Valeur minimale : {min(df['children'])}")
print(f"Valeur maximale : {max(df['children'])}")
print(f"Type de données : {df['children'].dtype}\n")

print("Observation : Les données sur le nombre d'enfants semblent correctes et ne contiennent pas de valeurs aberrantes.\n")

# Conclusion
print("Conclusion : Après ces observations, nous allons examiner plus en détail les valeurs de la colonne BMI.")


In [ ]:
bmi_outliers = valeurs_aberrantes_IQR(df, 'bmi')
bmi_outliers.sort_values('bmi', ascending=False)

In [ ]:
print(f"- Nombre de lignes contenant des valeurs abérantes : {bmi_outliers.shape[0]}")
print(f"- Pourgentage de lignes contenant des valeurs abérantes : {round((bmi_outliers.shape[0] / df.shape[0]) * 100,2)} %")

In [ ]:
df[df['bmi'] < 16]

In [ ]:
df[df['bmi'] > 40]

In [20]:
df.to_csv('dataset_assurance_cleaned.csv', index=False)

# 3 | Analyse univariée

In [ ]:
# Résumer statistiques de nos variables

summary_stats = df.describe().transpose()
summary_stats['median'] = df.select_dtypes('number').median()

summary_stats

### 1. Répartition des caractéristiques démographiques et sociales

In [ ]:
### Répartition des variables

fig, axes = plt.subplots(1, 4, figsize=(17, 4))

## Proportion Homme-Femmme
axes[0].pie(df["sex"].value_counts(), labels=df["sex"].value_counts().index, autopct='%1.1f%%')
axes[0].set_title("Homme - Femme")

## Proportion des fumeurs
axes[1].pie(df["smoker"].value_counts(), labels=df["smoker"].value_counts().index, autopct='%1.2f%%')
axes[1].set_title("Fumeur - Non-fumeur")

## Proportions des assurés par régions
axes[2].pie(df["region"].value_counts(), labels=df["region"].value_counts().index, autopct='%1.1f%%')
axes[2].set_title("Proportion des assurés par régions")

## Proportions des enfants par assurés
axes[3].pie(df["children"].value_counts(), labels=df["children"].value_counts().index, autopct='%1.1f%%')
axes[3].set_title("Proportion des enfants par assurés")


### 2. Distributions de nos variables

In [ ]:
### Distribution de notre variable cible 'charges'

sns.histplot(df['charges'], kde=True)

In [ ]:
### Distribution de nos variables


fig, axes = plt.subplots(1, 5, figsize=(20, 5))

# Graphique 1 : Distribution des âges
sns.histplot(df['age'], kde=True, ax=axes[0])
axes[0].set_title("Distribution des âges")
axes[0].set_xlabel("Age")
axes[0].set_ylabel("Nombre d'assurés")
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%.0f')

# Graphique 2 : Distribution du BMI
sns.histplot(df['bmi'], kde=True, ax=axes[1])
axes[1].set_title("Distribution des bmi")
axes[1].set_xlabel("BMI")
axes[1].set_ylabel("")
for container in axes[1].containers:
    axes[1].bar_label(container, fmt='%.0f')

# Graphique 3 : Distribution du nombre d'enfants
sns.histplot(df['children'], ax=axes[2])
axes[2].set_title("Distribution des enfants par assuré")
axes[2].set_xlabel("Children")
axes[2].set_ylabel("")
for container in axes[2].containers:
    axes[2].bar_label(container, fmt='%.0f')

# Graphique 4 : Distribution des assurés suivant s'ils sont fumeurs ou non-fumeurs
sns.histplot(df['smoker'], ax=axes[3])
axes[3].set_title("Distribution des assurés fumeurs ou non")
axes[3].set_xlabel("Smoker")
axes[3].set_ylabel("")
for container in  axes[3].containers:
    axes[3].bar_label(container, fmt='%.0f')

# Graphique 5 : Distribution des assurés par régions
sns.histplot(df['region'], ax=axes[4])
axes[4].set_title("Distriution des assurés par régions")
axes[4].set_xlabel("Region")
axes[4].set_ylabel("")
for container in axes[4].containers:
    axes[4].bar_label(container, fmt='%.0f')

plt.tight_layout()
plt.show()

# 4 | Corrélation des variables

In [ ]:
### Encodage de nos variables qualitatives pour le schéma de corrélations

encoder = LabelEncoder()

df_encoded = df.copy()

df_encoded['sex_encoded'] = encoder.fit_transform(df_encoded['sex'])
df_encoded['region_encoded'] = encoder.fit_transform(df_encoded['region'])
df_encoded['smoker_encoded'] =  encoder.fit_transform(df_encoded['smoker'])

df_encoded = df_encoded[['age', 'sex_encoded', 'bmi', 'children', 'smoker_encoded', 'region_encoded', 'charges']]
df_encoded

In [ ]:
### Heatmap des corrélations


# Corrélation
df_corr = df_encoded.select_dtypes('number').corr()

# HeatMap
sns.heatmap(df_corr, annot = True, fmt='.2f', cmap='seismic', center=0)


# 5 | Analyse Bivariés

In [ ]:
sns.pairplot(df.select_dtypes('number'))

In [ ]:
# Relation entre le sex et les charges

sns.barplot(data=df, x='sex', y='charges', estimator='mean', hue='sex', palette=['lightcoral', 'skyblue'], errorbar=None)

# Annotations
for container in plt.gca().containers:
    plt.gca().bar_label(container, fmt='%.0f$')

# Personnalisation
plt.xlabel("Sex")
plt.ylabel("Charges moyennes")
plt.title("Charges moyennes suivant le sex")
plt.show()


In [ ]:
# Relation entre le fait de fumer et les charges

## BarPlot
sns.barplot(data=df, x='smoker', y='charges', estimator='mean', errorbar=None)

## Annotations
for container in plt.gca().containers:
    plt.gca().bar_label(container, fmt='%.0f$')

#"" Personnalisation
plt.xlabel("Fumeurs")
plt.ylabel("Charges moyennes")
plt.title("Charges moyennes pour fumeurs et non-fumeurs")
plt.show()

In [ ]:
# Relation entre l'âge et les charges

df['age_group'] = pd.cut(
    df['age'],
    bins=[17, 24, 34, 44, 54, 64],                             
    labels=['18-24', '25-34', '35-44', '45-54', '55-64']
)

age_group_mapping = {'18-24': 0, '25-34': 1, '35-44': 2, '45-54': 3, '55-64': 4}
df['age_group_numeric'] = df['age_group'].map(age_group_mapping)

# BarPlot
sns.barplot(data=df, x='age_group', y='charges', estimator='mean', errorbar=None)

# Courbe de tendance avec regplot (régression linéaire)
sns.regplot(data=df, x='age_group_numeric', y='charges', scatter=False, color='red')

# Annotations
for container in plt.gca().containers:
    plt.gca().bar_label(container, fmt='%.0f$')

# Personnalisation
plt.xlabel("Groupe d'âges")
plt.ylabel("Charges moyennes")
plt.title("Charges moyennes par groupe d'âges avec courbe de tendance")
plt.show()

In [ ]:
pd.pivot_table(data=df, 
               values='charges', 
               index='age_group',   
               aggfunc=['count', 'mean', 'median', 'std'],
               observed=True
               ).round(2)

In [ ]:
# Relation entre le BMI et les charges

## Création des catégories de BMI
def bmi_category(bmi):
    if bmi < 18:
        return "Sous-poids"
    elif bmi < 24.5:
        return "Normal"
    elif bmi < 30:
        return "Surpoids"
    elif bmi < 35:
        return "Obésité I"
    elif bmi < 40:
        return "Obésité II"
    else:
        return "Obésité III"

df["bmi_cat"] = df["bmi"].apply(bmi_category)

## Mapping de BMI
bmi_mapping = {'Sous-poids': 0,'Normal': 1, 'Surpoids': 2, 'Obésité I': 3, 'Obésité II': 4, 'Obésité III': 5}

df['bmi_cat_numeric'] = df['bmi_cat'].map(bmi_mapping)

## BarPlot
sns.barplot(data=df, x='bmi_cat', y='charges', estimator="mean", errorbar=None, order=bmi_mapping.keys())

## Courbe de tendance avec regplot (régression linéaire)
sns.regplot(data=df, x='bmi_cat_numeric', y='charges', scatter=False, color='red')

## Annotations
for container in plt.gca().containers:
    plt.gca().bar_label(container, fmt='%.0f$')

## Personnalisation
plt.xlabel("Catégorie BMI")
plt.ylabel("Charges moyennes")
plt.title("Charges moyennes par catégorie BMI avec courbe de tendance")
plt.show()

In [ ]:
pd.pivot_table(data=df, 
               values='charges',  # On peut utiliser n'importe quelle colonne ici
               index='bmi_cat',   # C'est la colonne pour laquelle on veut les catégories
               aggfunc=['count', 'mean', 'median', 'std'],
               ).reindex(bmi_mapping).round(2)

In [ ]:
# Relation entre le nombre d'enfants et les charges

## BarPlot
sns.barplot(data=df, x='children', y='charges', errorbar=None)
sns.regplot(data=df, x='children', y='charges', scatter=False, color='red')

## Annotations
for container in plt.gca().containers:
    plt.gca().bar_label(container, fmt='%.0f$')

## Personnalisation
plt.xlabel("Nombre d'enfants")
plt.ylabel("Charges moyennes")
plt.title("Charges moyennes par nombre d'enfants avec courbe de tendance")
plt.show()

In [ ]:
pd.pivot_table(data=df, 
               values='charges',
               index='children',
               aggfunc=['count', 'mean', 'median', 'std']
               ).round(2)

In [ ]:
# Relation entre le nombre d'enfants et les charges

## BarPlot
sns.barplot(data=df, x='region', y='charges', estimator='mean', errorbar=None)

## Annotations
for container in plt.gca().containers:
    plt.gca().bar_label(container, fmt='%.0f$')

## Personnalisation
plt.xlabel("Régions")
plt.ylabel("Charges moyennes")
plt.title("Charges moyennes par région")
plt.show()

In [ ]:
pd.crosstab(df["sex"], df["region"])

In [ ]:
pd.crosstab(df["smoker"], df["region"])

In [ ]:
df.groupby(['sex', 'smoker'])['charges'].describe()

In [ ]:
df.groupby(['sex', 'region'])['charges'].describe()

In [ ]:
df.groupby(['smoker', 'region'])['charges'].describe()

# 6 | Tests statistiques

### 1. Test de normalités

In [ ]:
variables = ['charges', 'age', 'bmi']

results = []

# Test de normalité sur chaque variable
for var in variables:
    stat, p_value = stats.normaltest(df[var])
    results.append({
        'Variable': var,
        'Statistique': stat,
        'p-value': p_value,
        'Normalité': 'Rejetée' if p_value < 0.05 else 'Acceptée'
    })
    
# Ajouter les résultats après transformation logarithmique pour 'charges'
stat, p_value = stats.normaltest(np.log(df["charges"]))
results.append({
    'Variable': 'charges (log)',
    'Statistique': stat,
    'p-value': p_value,
    'Normalité': 'Rejetée' if p_value < 0.05 else 'Acceptée'
})

norm_test_df = pd.DataFrame(results)
norm_test_df


### 2. Tests de régression univariée

In [ ]:
# Charges & Âges

print(smf.ols('charges ~ age', data=df).fit().summary())

Le modèle n'explique qu'environ 8.9 % de la variance des charges, ce qui est très faible. Cela signifie que d'autres variables (comme bmi, smoker, ou sex) sont probablement importantes pour expliquer les variations des charges.

Bien que age ait un effet significatif sur les charges, cet effet est relativement faible (257.23 unités monétaires par an).

Les résidus ne sont pas normalement distribués, ce qui pourrait affecter la validité des tests statistiques. Cela peut être dû à l'absence d'autres variables importantes dans le modèle.

In [ ]:
# Charges & BMI

print(smf.ols('charges ~ bmi', data=df).fit().summary())

R² = 0.039 : 3.9 % de la variabilité des charges est expliquée par l’IMC.
Le coefficient (393.86) montre qu’une augmentation d’une unité de l’IMC est associée à une augmentation moyenne de 393.86 unités monétaires des charges.

Le test global du modèle (F−statistic = 54.70) est également significatif (P=2.47×10^(−13)), confirmant que bmi apporte une information utile pour expliquer charges.

Bien que bmi ait un effet significatif sur charges, il ne suffit pas à expliquer les variations importantes des coûts médicaux. 

In [ ]:
# Fumeurs

print(smf.ols('charges ~ smoker', data=df).fit().summary())

R² = 0.62 : Cela signifie que 62 % de la variation des charges médicales est expliquée par la variable explicative "smoker" (le statut de fumeur).
Ce résultat indique une bonne capacité explicative du modèle. Un R² aussi élevé montre que le statut de fumeur a un impact significatif sur les charges médicales.

smoker[T.yes] = 23 610,00 : Cela signifie que les charges médicales sont en moyenne 23 610 unités monétaires plus élevées pour un fumeur comparé à un non-fumeur.

F-statistic = 2176 et p-value associée = 1.41e-282 : Le test F est très significatif (p-value extrêmement faible), ce qui confirme que le modèle dans son ensemble est bien ajusté et que la variable explicative "smoker" contribue de manière importante à la prédiction des charges.

In [ ]:
# Charges & Enfants

print(smf.ols('charges ~ children', data=df).fit().summary())

R² = 0.005 : Cela signifie que seulement 0,5 % de la variation des charges médicales est expliquée par le nombre d'enfants. Un R² aussi faible suggère que le modèle n'explique pratiquement rien de la variation des charges.

In [ ]:
# Regions

print(smf.ols('charges ~ region', data=df).fit().summary())

R² = 0.007 : Cela suggère que les variables indépendantes incluses dans le modèle n'ont qu'une influence marginale sur charges.

Variables catégorielles "region" :
* "northwest" : Le coefficient est de -955,54, mais avec une valeur p de 0,314, ce qui indique que cette variable n'a pas d'effet statistiquement significatif sur "charges".
* "southeast" : Le coefficient est de 1 329,03, avec une valeur p de 0,150, suggérant également un effet non significatif.
* "southwest" : Le coefficient est de -1 059,45, avec une valeur p de 0,264, indiquant un effet non significatif.
Aucune des variables "region" n'a d'effet statistiquement significatif sur "charges" dans ce modèle.


F-statistic : La valeur est de 2,926 avec une valeur p de 0,0328, indique que le modèle dans son ensemble est significatif. Bien que le modèle global soit statistiquement significatif, les variables régionales incluses n'ont pas d'effet significatif sur charges. De plus, les résidus ne suivent pas une distribution normale, ce qui pourrait affecter la validité des résultats.

### 3. Test de régression multivarié

In [ ]:
df_encoded

In [ ]:
X = df_encoded.drop(columns='charges')
y = df['charges']

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

print(model.summary())

Les résultats de la régression linéaire multiple indiquent que le modèle explique environ 75,1 % de la variance des charges médicales, ce qui est relativement élevé. Les variables indépendantes significatives comprennent l'âge, l'indice de masse corporelle (IMC), le nombre d'enfants, le statut de fumeur et la région. En revanche, le sexe n'a pas d'effet statistiquement significatif sur les charges médicales dans ce modèle.

R² = 0,751 : Le modèle explique environ 75,1% de la variance de la variable dépendante "charges". Cela signifie que le modèle est relativement performant.
Adj. R² = 0,749 : L'ajustement prend en compte le nombre de variables explicatives, ce qui reste très bon. L'ajustement montre qu'il y a très peu de perte de performance lorsque l'on ajuste pour la complexité du modèle.

F-statistic = 667,0 avec une p-value proche de 0 : Cela signifie que le modèle dans son ensemble est très significatif et que la probabilité que les résultats soient dus au hasard est extrêmement faible.


* **age** : Le coefficient est 257,20, avec une p-value = 0,000. Cela montre que l'âge a un effet statistiquement significatif sur les charges (une augmentation de l'âge de 1 an est associée à une augmentation de 257,20 unités des charges).

* **sex_encoded** : Le coefficient est de -129,40, mais avec une p-value = 0,698. Cela indique que le sexe n'a pas un effet significatif sur les charges.

* **bmi** : Le coefficient est 332,60, avec une p-value = 0,000. Cela montre que le BMI a un impact statistiquement significatif sur les charges (une augmentation du BMI de 1 unité est associée à une augmentation de 332,60 unités des charges).

* **children** : Le coefficient est 478,77, avec une p-value = 0,001. Cela montre qu'avoir des enfants a un effet positif et significatif sur les charges (chaque enfant supplémentaire est associé à une augmentation de 478,77 unités des charges).

* **smoker_encoded** : Le coefficient est de 23 820, avec une p-value = 0,000. Cela montre qu'être fumeur a un impact majeur et significatif sur les charges.

* **region_encoded** : Le coefficient est de -354,01, avec une p-value = 0,020. Cela montre que la région a un effet modéré mais significatif sur les charges (certaines régions entraînent une réduction ou une augmentation des charges par rapport à la moyenne).